In [6]:
import torch
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from torchvision.models import resnet50
from torch.nn import Linear, CrossEntropyLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

MODEL_PATH = "../model/ResNet50_cifar100_v3.pth"
# MODEL_PATH = "../model/HW4_ResNet50.pth"

cuda


In [7]:

# Import EfficientNet and load self-trained weights
# from efficientnet_pytorch import EfficientNet

# Define the data transformations for testing
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# test_transform = transforms.Compose([
#     transforms.Resize(256),
#     transforms.CenterCrop(224),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# Set the path to the directory containing the test data
test_data_dir = "../training"

# Load the test data
test_dataset = ImageFolder(test_data_dir, transform=test_transform)
test_dl = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
# Model
# PATH = "../model/ResNet50_cifar100.pth"
model = resnet50()

last_fc = model.fc.in_features
model.fc = Linear(last_fc, 10)

model.load_state_dict(torch.load(MODEL_PATH))


model = model.to(device)
# Set the model to evaluation mode
model.eval()
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:

# Lists to store the predicted labels and ground truth labels
all_preds = []
all_labels = []

# Iterate over the test dataset and collect predicted labels and ground truth labels
with torch.no_grad():
    for xtest, ytest in test_dl:
        xtest = xtest.to(device)
        test_prob = model(xtest)
        test_prob = test_prob.cpu()

        test_pred = torch.max(test_prob, 1).indices
        all_preds.extend(test_pred.numpy())
        all_labels.extend(ytest.numpy())


In [10]:

# Create a confusion matrix
confusion_mat = confusion_matrix(all_labels, all_preds)

# Plot the confusion matrix as a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mat, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.savefig("../out/ResNet50/gray_resnet50_confusion_matrix_teacher_v3.png")  # Save the confusion matrix as an image file
plt.close()

# Calculate accuracy
accuracy = np.sum(np.diag(confusion_mat)) / np.sum(confusion_mat)

# Save the accuracy as a text file
with open("../out/ResNet50/gray_resnet50_accuracy_teacher_v3", "w") as file:
    file.write(f"Accuracy: {accuracy}")

# Print the accuracy
print(f"Accuracy: {accuracy}")

Accuracy: 0.9898166195950515
